## 加载数据

In [286]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

class Data_loader(Dataset):
    def __init__(self, data_frame, img_dir, transform=None, is_test=False):
        """
        自定义的 PyTorch Dataset，用于加载图像和相关的标签数据。

        参数:
        - data_frame (pandas.DataFrame): 包含图像文件名和标签的 DataFrame。
        - img_dir (str): 图像文件夹路径。
        - transform (callable, optional): 图像变换（数据增强等）。
        - is_test (bool): 是否为测试集标志。如果是测试集，没有标签。
        """
        self.data_frame = data_frame  # 直接使用传入的DataFrame
        self.img_dir = img_dir  # 图像文件夹路径
        self.transform = transform  # 图像变换
        self.is_test = is_test  # 是否是测试集

    def __len__(self):
        """返回数据集的大小"""
        return len(self.data_frame)

    def __getitem__(self, idx):
        """根据索引获取图像和标签"""
        img_name = os.path.join(self.img_dir, str(self.data_frame.iloc[idx, 0]) + ".jpg")  # 假设第一列是图像文件名
        image = Image.open(img_name).convert("RGB")  # 打开图像并转换为RGB格式

        if self.is_test:
            if self.transform:
                image = self.transform(image)
            return image  # 测试集没有标签，返回图像即可
        else:
            label = self.data_frame['stable_height'].iloc[idx]  # 使用 stable_height 作为标签
            if self.transform:
                image = self.transform(image)
            return image, label



In [287]:
# 定义图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 加载并划分训练数据
train_csv_path = 'data/train.csv'
train_data = pd.read_csv(train_csv_path)

# 使用 train_test_split 划分训练集和验证集
train_df, val_df = train_test_split(train_data, test_size=0.99, random_state=42)

# 创建训练集和验证集的数据加载器
train_img_dir = 'data/train/'
train_dataset = Data_loader(train_df, train_img_dir, transform=transform)
val_dataset = Data_loader(val_df, train_img_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 加载测试数据
test_csv_path = 'data/test.csv'
test_img_dir = 'data/test/'
test_data = pd.read_csv(test_csv_path)
test_dataset = Data_loader(test_data, test_img_dir, transform=transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [312]:
from torch.utils.data import Subset
# 假设你只想取前1000个验证集样本
subset_indices = list(range(10))  # 选择前1000个样本的索引
val_subset = Subset(val_dataset, subset_indices)

# 创建新的 DataLoader
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

# 打印新的验证集数量
print(f"新的验证集数量: {len(val_subset)}")

新的验证集数量: 10


In [313]:
print(f"训练集数量: {len(train_dataset)}")
print(f"验证集数量: {len(val_dataset)}")
print(f"测试集数量: {len(test_dataset)}")

训练集数量: 76
验证集数量: 7604
测试集数量: 1920


In [289]:
# # 从训练数据加载器中取出一个 batch
# data_iter = iter(train_loader)
# images, labels = next(data_iter)

# # 打印批次中的图像和标签
# print(f"图像批次大小: {images.size()}")  # 图像的张量大小，例如 [32, 3, 128, 128] 表示32张图像，每张图像是3通道，128x128
# print(f"标签: {labels}")  # 打印标签

In [290]:
# import matplotlib.pyplot as plt
# import numpy as np

# # 可视化一个批次的图像
# def imshow(img):
#     img = img / 2 + 0.5  # 将图像还原
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# # 获取一个批次的数据
# data_iter = iter(train_loader)
# images, labels = next(data_iter)

# # 显示图像
# imshow(images[0])  # 显示批次中的第一张图像
# print(f"标签: {labels[0]}")  # 打印第一张图像的标签

## CNN 模型
层数 3
kernel numbers: 32 64 128
kernel size: 3 x 3
active function: relu

In [291]:
import torch
import torch.nn as nn

class CNN_Model(nn.Module):
    def __init__(self):
        super(CNN_Model, self).__init__()
        # 定义3层卷积层和最大池化层
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        # 修改后的全连接层
        self.fc1 = nn.Linear(128 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        
        # Dropout层用于避免过拟合
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        
        # 展平输入
        x = x.view(x.size(0), -1)  # 动态展平，确保批次大小正确
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        
        # print(f"模型输出的形状: {x.shape}")  # 打印输出的形状
        return x

## 使用预训练的 ResNet50 模型

In [292]:
import torch
import torch.nn as nn
import torchvision.models as models

class ResNet50_Model(nn.Module):
    def __init__(self):
        super(ResNet50_Model, self).__init__()
        # 加载预训练的 ResNet50 模型
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        num_ftrs = self.resnet.fc.in_features
        
        # 替换 ResNet50 的全连接层
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_ftrs, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # 最终输出一个标量，用于回归任务
        )
    
    def forward(self, x):
        # print(f"模型输出的形状: {x.shape}")  # 打印输出的形状
        return self.resnet(x)

In [293]:
import torch.nn as nn
import torchvision.models as models

class ResNeXt101(nn.Module):
    def __init__(self):
        super(ResNeXt101, self).__init__()
        # 加载预训练的 ResNeXt101 模型
        self.resnet = models.resnext101_64x4d(weights=models.ResNeXt101_64X4D_Weights.IMAGENET1K_V1)
        num_ftrs = self.resnet.fc.in_features
        # 替换全连接层
        self.resnet.fc = nn.Identity()

        # 定义新的全连接层
        self.fc = nn.Sequential(
            nn.Linear(num_ftrs, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1)  # 输出单个标量
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        # print(f"模型输出的形状: {x.shape}")  # 打印输出的形状
        return x
    
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        # 加载预训练的 ResNet18 模型
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        num_ftrs = self.resnet.fc.in_features
        # 替换 ResNet18 的全连接层
        self.resnet.fc = nn.Identity()
        
        # 定义新的全连接层
        self.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # 输出单个标量
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        # print(f"模型输出的形状: {x.shape}")  # 打印输出的形状
        return x
        
# 据说这个最好    
class GoogLeNet(nn.Module):
    def __init__(self):
        super(GoogLeNet, self).__init__()
        # 加载预训练的 GoogLeNet 模型
        self.googlenet = models.googlenet(weights=models.GoogLeNet_Weights.IMAGENET1K_V1)
        num_ftrs = self.googlenet.fc.in_features
        # 替换 GoogLeNet 的全连接层
        self.googlenet.fc = nn.Identity()

        # 定义新的全连接层
        self.fc = nn.Sequential(
            nn.Linear(num_ftrs, 1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 1)  # 输出单个标量
        )

    def forward(self, x):
        x = self.googlenet(x)
        x = self.fc(x)
        # print(f"模型输出的形状: {x.shape}")  # 打印输出的形状
        return x
    
class ViT(nn.Module):
    def __init__(self):
        super(ViT, self).__init__()
        # 加载预训练的 ViT 模型
        self.vit = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
        num_ftrs = self.vit.heads.head.in_features
        # 替换 ViT 的全连接层
        self.vit.heads = nn.Identity()

        # 定义新的全连接层
        self.fc = nn.Sequential(
            nn.Linear(num_ftrs, 1024),
            nn.ReLU(),
            nn.Linear(1024, 125),
            nn.BatchNorm1d(125),  # 添加正则化层
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(125, 1)  # 将输入调整为 125 以匹配上面的输出
        )

    def forward(self, x):
        x = self.vit(x)
        x = self.fc(x)
        # print(f"模型输出的形状: {x.shape}")  # 打印输出的形状
        return x

## 训练模型

In [306]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import os

class ModelTrainer:
    def __init__(self, model, train_loader, val_loader,model_name, criterion=None, optimizer=None, num_epochs=10, learning_rate=0.001):
        """
        初始化训练类，准备模型、数据和相关参数

        参数:
        - model: 待训练的模型
        - train_loader: 训练数据的 DataLoader
        - val_loader: 验证数据的 DataLoader
        - criterion: 损失函数
        - optimizer: 优化器
        - num_epochs: 训练的 epoch 数
        - learning_rate: 学习率
        """
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.model_name = model_name
        self.criterion = criterion if criterion else nn.MSELoss() # MSEloss 作为损失函数
        # self.criterion = nn.CrossEntropyLoss() # CrossEntropyLoss 作为损失函数
        self.optimizer = optimizer if optimizer else optim.Adam(model.parameters(), lr=learning_rate)
        self.num_epochs = num_epochs
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

    def train_one_epoch(self, epoch):
        """
        训练一个 epoch 的数据，并返回训练损失
        """
        self.model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(self.train_loader, desc=f'Epoch {epoch+1}/{self.num_epochs}', unit="batch"):
            inputs, labels = inputs.to(self.device), labels.to(self.device).float()

            # 前向传播
            self.optimizer.zero_grad()
            outputs = self.model(inputs)
            loss = self.criterion(outputs.squeeze(), labels)

            # 反向传播与优化
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()
            # print("one epoch----")
        avg_loss = running_loss / len(self.train_loader)
        # print("one epoch end----")
        return avg_loss

    def validate(self):
        """
        在验证集上进行评估，返回验证损失
        """
        # print("validate start----")
        self.model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in self.val_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device).float()
                outputs = self.model(inputs)
                loss = self.criterion(outputs.squeeze(), labels)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(self.val_loader)
        # print("validate end----")
        return avg_val_loss

    def train(self):
        """
        训练模型，并在每个 epoch 结束后进行验证
        """
        log_dir = self.create_log_dir()  # 创建日志文件夹
        writer = SummaryWriter(log_dir)

        best_val_loss = float('inf')
        for epoch in range(self.num_epochs):
            # 训练一个 epoch
            train_loss = self.train_one_epoch(epoch)
            val_loss = self.validate()

            print(f'Epoch [{epoch+1}/{self.num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
            
            # 记录损失值
            writer.add_scalar('Loss/Train', train_loss, epoch)
            writer.add_scalar('Loss/Validation', val_loss, epoch)
            # print("train epoch----")
            # 保存最好的模型
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                # print("save------")
                self.save_model(f'models/best_model_{self.model_name}.pth')
        # print("train end -----")
        writer.close()

    def create_log_dir(self):
        """
        创建日志文件夹
        """
        from datetime import datetime
        current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        log_dir = f'runs/experiment_{current_time}'
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        return log_dir

    def save_model(self, model_path):
        """
        保存模型到指定路径
        """
        if not os.path.exists('models'):
            os.makedirs('models')
        torch.save(self.model.state_dict(), model_path)
        print(f"模型已保存到 {model_path}")

    

In [307]:
# model = CNN_Model()  # 例如 CNN 模型
# trainer = ModelTrainer(model, train_loader, val_loader,'CNN_Model', num_epochs=1, learning_rate=0.001)
# trainer.train()

In [314]:
# model = ResNeXt101() 
# trainer = ModelTrainer(model, train_loader, val_loader,'ResNeXt101', num_epochs=1, learning_rate=0.001)
# trainer.train()

Epoch 1/1:  33%|███▎      | 1/3 [00:27<00:55, 27.83s/batch]

one epoch----


Epoch 1/1:  67%|██████▋   | 2/3 [00:54<00:27, 27.30s/batch]

one epoch----


Epoch 1/1: 100%|██████████| 3/3 [00:57<00:00, 19.08s/batch]

one epoch----
one epoch end----
validate start----


validate end----
Epoch [1/1], Train Loss: 6.9639, Validation Loss: 2.5151
train epoch----
save------
模型已保存到 models/best_model_ResNeXt101.pth
train end -----


In [315]:
# model = ResNet18() 
# trainer = ModelTrainer(model, train_loader, val_loader,'ResNet18', num_epochs=1, learning_rate=0.001)
# trainer.train()

Epoch 1/1:  33%|███▎      | 1/3 [00:01<00:03,  1.85s/batch]

one epoch----


Epoch 1/1:  67%|██████▋   | 2/3 [00:03<00:01,  1.83s/batch]

one epoch----


Epoch 1/1: 100%|██████████| 3/3 [00:03<00:00,  1.33s/batch]

one epoch----
one epoch end----
validate start----
validate end----
Epoch [1/1], Train Loss: 6.6761, Validation Loss: 5.0824
train epoch----
save------
模型已保存到 models/best_model_ResNet18.pth
train end -----


In [316]:
# model = GoogLeNet() 
# trainer = ModelTrainer(model, train_loader, val_loader,'GoogLeNet', num_epochs=1, learning_rate=0.001)
# trainer.train()

Epoch 1/1:  33%|███▎      | 1/3 [00:01<00:03,  1.99s/batch]

one epoch----


Epoch 1/1:  67%|██████▋   | 2/3 [00:03<00:01,  1.96s/batch]

one epoch----


Epoch 1/1: 100%|██████████| 3/3 [00:04<00:00,  1.42s/batch]

one epoch----
one epoch end----
validate start----
validate end----
Epoch [1/1], Train Loss: 5.9321, Validation Loss: 11.5069
train epoch----
save------
模型已保存到 models/best_model_GoogLeNet.pth
train end -----


In [317]:
# model = ViT() 
# trainer = ModelTrainer(model, train_loader, val_loader,'ViT', num_epochs=1, learning_rate=0.001)
# trainer.train()

Epoch 1/1:  33%|███▎      | 1/3 [00:05<00:10,  5.28s/batch]

one epoch----


Epoch 1/1:  67%|██████▋   | 2/3 [00:09<00:04,  4.53s/batch]

one epoch----


Epoch 1/1: 100%|██████████| 3/3 [00:10<00:00,  3.54s/batch]

one epoch----
one epoch end----
validate start----


validate end----
Epoch [1/1], Train Loss: 11.2113, Validation Loss: 8.6611
train epoch----
save------
模型已保存到 models/best_model_ViT.pth
train end -----


In [ ]:
def train_all_models(models, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    """
    训练传入的所有模型，并保存每个模型的最佳权重。

    参数:
    - models: 一个包含所有模型结构的字典，key 为模型名称，value 为模型实例。
    - train_loader: 训练数据的 DataLoader。
    - val_loader: 验证数据的 DataLoader。
    - num_epochs: 每个模型训练的 epoch 数。
    - learning_rate: 学习率。
    """
    for model_name, model in models.items():
        print(f"正在训练模型: {model_name}")
        trainer = ModelTrainer(model, train_loader, val_loader, model_name, num_epochs=num_epochs, learning_rate=learning_rate)
        trainer.train()
        print(f"模型 {model_name} 训练完成并保存。")

# 使用字典来存储你要训练的模型
models = {
    'CNN_Model': CNN_Model(),
    'ResNet50_Model': ResNet50_Model(),
    'ResNeXt101': ResNeXt101(),
    'ResNet18': ResNet18(),
    'GoogLeNet': GoogLeNet(),
    'ViT': ViT(),
}

# 调用函数来训练所有的模型
train_all_models(models, train_loader, val_loader, num_epochs=1, learning_rate=0.001)


## 预测

In [318]:
import torch
import pandas as pd

class Predictor:
    def __init__(self, model, model_path, device=None):
        """
        初始化预测类，加载模型

        参数:
        - model: 要加载的模型结构（未加载权重）
        - model_path: 保存的模型权重路径
        - device: 使用的设备（默认为 None，将自动选择 CPU 或 GPU）
        """
        self.device = device if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.model.load_state_dict(torch.load(model_path))  # 加载模型权重
        self.model.eval()  # 设置模型为评估模式
        print(f"模型已从 {model_path} 加载")

    def predict(self, test_loader, output_file='predictions/submission.csv'):
        """
        使用训练好的模型对测试集进行预测，并生成提交文件。

        参数:
        - test_loader: 测试数据加载器
        - output_file: 生成的提交文件名
        """
        predictions = []
        ids = []
        with torch.no_grad():
            for inputs in test_loader:
                inputs = inputs.to(self.device)
                outputs = self.model(inputs)
                predicted_heights = outputs.squeeze().cpu().numpy()  # 将预测结果转为 numpy 数组
                predictions.extend(predicted_heights)

        # 读取测试集的id并生成提交文件
        test_csv = pd.read_csv('data/test.csv')
        submission = pd.DataFrame({
            'id': test_csv['id'],
            'stable_height': predictions
        })
        submission.to_csv(output_file, index=False)
        print(f"提交文件已生成: {output_file}")


In [321]:
# 假设你已经有一个训练好的模型 CNN_Model() 和 test_loader
model = ViT()  # 你的模型结构

# 初始化 Predictor 类，传入模型和模型权重路径
predictor = Predictor(model=model, model_path='models/best_model_ViT.pth')

# 使用 predictor 对测试集进行预测并生成提交文件
predictor.predict(test_loader=test_loader, output_file='predictions/submission.csv')


/var/folders/sm/cm_br_015_9864d7n_0rkhsm0000gn/T/ipykernel_68895/3926736127.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_

模型已从 models/best_model_ViT.pth 加载
提交文件已生成: submission.csv


In [ ]:

# 定义模型和对应的模型路径
models_info = {
    'CNN_Model': ('models/best_model_CNN_Model.pth', CNN_Model()),
    'ResNet50_Model': ('models/best_model_ResNet50_Model.pth', ViT()),
    'ViT': ('models/best_model_ViT.pth', ViT()),  # ViT 模型
    'ResNet18': ('models/best_model_ResNet18.pth', ResNet18()),  # ResNet18 模型
    'ResNeXt101': ('models/best_model_ResNeXt101.pth', ResNeXt101()),  # ResNeXt101 模型
    'GoogLeNet': ('models/best_model_GoogLeNet.pth', GoogLeNet()),  # GoogLeNet 模型
    # 你可以继续添加其他模型
}

# 创建一个函数，使用 Predictor 进行预测
def run_prediction_for_models(models_info, test_loader, output_folder='predictions'):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for model_name, (model_path, model_instance) in models_info.items():
        print(f"正在预测模型: {model_name}")
        
        # 初始化 Predictor 类，传入模型和模型权重路径
        predictor = Predictor(model=model_instance, model_path=model_path)
        
        # 使用 predictor 对测试集进行预测并生成提交文件
        output_file = f"{output_folder}/submission_{model_name}.csv"
        predictor.predict(test_loader=test_loader, output_file=output_file)
        
        print(f"模型 {model_name} 的预测完成并保存至: {output_file}")

# 假设你已经有 test_loader
run_prediction_for_models(models_info, test_loader)

## 数据增强

In [257]:
# 定义图像预处理和数据增强
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomRotation(20),  # 随机旋转20度以内
    transforms.RandomResizedCrop(224),  # 随机裁剪并调整为224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 调整亮度、对比度、饱和度、色调
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

# 验证集和测试集通常不需要数据增强，只需要对数据进行预处理
transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),  # 直接调整图像大小
    transforms.ToTensor(),  # 转换为张量
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 归一化
])

# 加载并划分训练数据
train_csv_path = 'data/train.csv'
train_data = pd.read_csv(train_csv_path)

# 使用 train_test_split 划分训练集和验证集
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

# 创建训练集和验证集的数据加载器
train_img_dir = 'data/train/'
train_dataset = Data_loader(train_df, train_img_dir, transform=transform_train)  # 使用数据增强的transform
val_dataset = Data_loader(val_df, train_img_dir, transform=transform_val_test)  # 验证集使用不带数据增强的transform

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 加载测试数据
test_csv_path = 'data/test.csv'
test_img_dir = 'data/test/'
test_data = pd.read_csv(test_csv_path)
test_dataset = Data_loader(test_data, test_img_dir, transform=transform_val_test, is_test=True)  # 测试集使用同样的transform
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [258]:
# model = CNN_Model()  # 例如 CNN 模型
# trainer = ModelTrainer(model, train_loader, val_loader,'CNN_Model', num_epochs=1, learning_rate=0.001)
# trainer.train()

Epoch 1/1: 100%|██████████| 192/192 [02:35<00:00,  1.23batch/s]


Epoch [1/1], Train Loss: 302.9137, Validation Loss: 313.5047
模型已保存到 models/best_model_CNN_Model.pth


In [ ]:
def train_all_models(models, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    """
    训练传入的所有模型，并保存每个模型的最佳权重。

    参数:
    - models: 一个包含所有模型结构的字典，key 为模型名称，value 为模型实例。
    - train_loader: 训练数据的 DataLoader。
    - val_loader: 验证数据的 DataLoader。
    - num_epochs: 每个模型训练的 epoch 数。
    - learning_rate: 学习率。
    """
    for model_name, model in models.items():
        print(f"正在训练模型: {model_name}")
        trainer = ModelTrainer(model, train_loader, val_loader, model_name, num_epochs=num_epochs, learning_rate=learning_rate)
        trainer.train()
        print(f"模型 {model_name} 训练完成并保存。")

# 使用字典来存储你要训练的模型
models = {
    'CNN_Model': CNN_Model(),
    'ResNet50_Model': ResNet50_Model(),
    'ResNeXt101': ResNeXt101(),
    'ResNet18': ResNet18(),
    'GoogLeNet': GoogLeNet(),
    'ViT': ViT(),
}

# 调用函数来训练所有的模型
train_all_models(models, train_loader, val_loader, num_epochs=1, learning_rate=0.001)


In [ ]:

# 定义模型和对应的模型路径
models_info = {
    'CNN_Model': ('models/best_model_CNN_Model.pth', CNN_Model()),
    'ResNet50_Model': ('models/best_model_ResNet50_Model.pth', ViT()),
    'ViT': ('models/best_model_ViT.pth', ViT()),  # ViT 模型
    'ResNet18': ('models/best_model_ResNet18.pth', ResNet18()),  # ResNet18 模型
    'ResNeXt101': ('models/best_model_ResNeXt101.pth', ResNeXt101()),  # ResNeXt101 模型
    'GoogLeNet': ('models/best_model_GoogLeNet.pth', GoogLeNet()),  # GoogLeNet 模型
    # 你可以继续添加其他模型
}

# 创建一个函数，使用 Predictor 进行预测
def run_prediction_for_models(models_info, test_loader, output_folder='predictions'):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for model_name, (model_path, model_instance) in models_info.items():
        print(f"正在预测模型: {model_name}")
        
        # 初始化 Predictor 类，传入模型和模型权重路径
        predictor = Predictor(model=model_instance, model_path=model_path)
        
        # 使用 predictor 对测试集进行预测并生成提交文件
        output_file = f"{output_folder}/submission_{model_name}.csv"
        predictor.predict(test_loader=test_loader, output_file=output_file)
        
        print(f"模型 {model_name} 的预测完成并保存至: {output_file}")

# 假设你已经有 test_loader
run_prediction_for_models(models_info, test_loader)